In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# 소개
샌프란시스코는 세계에서 가장 악명 높은 범죄자들을 수용한 탈출 불가능한 알카트라즈 (Ancatraz) 섬으로 악명이 높았다. 오늘날 이 도시는 테크 산업으로 더 많이 알려져 있습니다. 이 프로젝트의 목적은 Sunset에서 SOMA까지, Marina에서 Excelsior까지, 샌프란시스코의 모든 지역에서 발생한 12 년간의 범죄를 분석하여 발생 시간과 위치 정보를 통해 범죄 유형을 예측하는 모델을 만드는 것입니다.
  
# 정의  
## 프로젝트 개요  

범죄는 사회 자체만큼이나 오래된 사회 현상이고, 범죄로부터 자유로운 사회는 없지만, - 사회의 모든 사람들이 같은 방식으로 사고하고 행동해야 하기 때문에 - 사회는 항상 그것을 방지하고 최소화하는 방법을 모색합니다.
근대 미국 역사에서 범죄율은 2차대전 이후부터 증가해 왔고, 1970 년대부터 1990 년대 초반까지 피크를 찍었습니다. 폭력 범죄는 1960년과 1991년 사이에 4배나 증가했습니다. 재산 범죄는 같은 기간 동안 두 배 이상 증가했습니다. 그러나 1990 년대 이래 미국의 범죄는 꾸준히 감소했다.
최근까지도 범죄 예방은 엄격한 행동 및 사회적 방법을 기반으로 연구되었지만, 데이터 분석 방법의 발전은 보다 정량적인 접근을 가능하게 했습니다. 우리는 샌프란시스코의 모든 지역에서 발생한 12년간의 범죄 데이터 세트를 살펴보고, 시간과 장소를 따른 발생하는 범죄의 범주를 예측하는 모델을 만들 것입니다.

## 문제 설명
문제를 검토하기 위해 다음과 같이 구성된 'full Data Science life cycle'를 적용합니다 :
1. 데이터의 품질을 감사하고 데이터셋을 청소하기 위한 모든 행동을 수행하기 위한 논의
2. 변수와 데이터를 이해하기 위한 데이터 탐색
3. 기존의 변수로부터 새 변수를 만들기 위한 Feature Engineering.
4. 학습 알고리즘에 사용할 데이터셋에 Normalization 과 Transformation 사용(필요시).
5. 모델의 성능을 평가하고 모델의 하이퍼파라메터를 fine-tune 하기 위한 training/test 데이터 생성
6. 모델 선택과 평가. 이것이 최종 목표임; 범죄가 발생한 시간과 장소로 범죄 타입을 에측하는 모델을 생성

## Metrics
이러한 문제에 가장 적합한 evaluation metrics 는 **multi-class logarithmic loss** 입니다. Logarithmic loss 는 출력이 0과 1 사이의 확률 값인 classification model의 성능을 평가한다.
For each incident, we will predict a set of predicted probabilities (one for every class), and we will calculate the average deviation from the real values. 
각 사건별로 예상 확률(모든 사건에 대해)을 예측하고 실제 값과의 평균 편차를 계산합니다.
To get a little more intuition on the metric, for a specific incident:
측정 항목에 대해 좀 더 잘 이해하기 위해 특정 사건에 대해서는,
* 계산한 예측 확률과 상관없이, 발생하지 않은 사건에 대해서는 loss를 0으로 처리합니다. (yij =0 yijlog(pij) 이므로)
* 발생한 확률에 대해선 log(pij) 의 loss값을 갖는다. 여기서 pij는 특정 카테고리에 대한 예측 확률 입니다.
특정 사건이 어떤 카테고리에 속하는지의 확률 총합은 1이므로, 일어나지 않은 카테고리에 대해 예측한 확률은 발생한 카테고리에 대해 계산한 확률값을 감소시키는 "간접적인" loss를 발생시킨다.


In other words, the metric evaluates the certainty of our model for each category of crime/incident.  

다시 말해 metric은 범죄/사건의 각 카테고리에 대한 모델의 'certainty'(확실성)를 평가한다.

# 분석  
## 데이터 탐색  
이 dataset은 표 형식이고 시간 순서의 지리적/텍스트 데이터를 포함하고 SFPD 범죄사건 보고 시스템으로부터 얻은 사건 정보들을 포함한다.

In [ ]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
import contextily as ctx
import geoplot as gplt
import lightgbm as lgb
import eli5
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots
import shap

In [ ]:
train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')

In [ ]:
print('First date: ', str(train.Dates.describe()['first']))
print('Last date: ', str(train.Dates.describe()['last']))
print('Test data shape ', train.shape)

1/1/2003 부터 5/13/2015까지의 데이터는 9개의 속성을 가진 878,049개의 데이터 샘플을 이루고 있다.

In [ ]:
train.head()

자세히 살펴보면, 각각의 데이터 샘플들은 다음의 feature로 이루어져 있다.
* Dates - 범죄 사건이 발생한 시간.
* Category - 범죄 사건의 종류(카테고리) (target variable)
* Descript - 범죄 사건의 세부 내용
* DayOfWeek - 발생한 시간의 요일
* PdDistrict - 발생한 지역을 담당하는 경찰서 지구 이름
* Resolution - 처리(결과) 내용
* Address - 발생한 위치의 대략적인 주소
* X - 경도
* Y - 위도


In [ ]:
train.dtypes

dataset에서는 인코딩 처리해야 할 오브젝트 변수(일명 문자열)가 많이 포함되어 있다.

In [ ]:
train.duplicated().sum()

또한 2323개의 중복된 데이터를 제거해야 한다.
또한 우리는 X, Y 로 이루어진 좌표값을 검증할 것이다.

In [ ]:
def create_gdf(df):
    gdf = df.copy()
    gdf['Coordinates'] = list(zip(gdf.X, gdf.Y))
    gdf.Coordinates = gdf.Coordinates.apply(Point)
    gdf = gpd.GeoDataFrame(
        gdf, geometry='Coordinates', crs={'init': 'epsg:4326'})
    return gdf

train_gdf = create_gdf(train)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(color='white', edgecolor='black')
train_gdf.plot(ax=ax, color='red')
plt.show()

Some points are misplaced. Let's see how many they are.
몇몇 좌표의 위치가 잘못되어 있다. 잘못된 좌표의 갯수를 확인한다.

In [ ]:
print(train_gdf.loc[train_gdf.Y > 50].count()[0])
train_gdf.loc[train_gdf.Y > 50].sample(5)

정상에서 벗어난 이상치 데이터의 좌표값을 동일한 경찰서 지구('PdDistrict')를 가진 데이터의 평균값으로 교체한다.

In [ ]:
train.drop_duplicates(inplace=True)
train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

imp = SimpleImputer(strategy='mean')

for district in train['PdDistrict'].unique():
    train.loc[train['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(
        train.loc[train['PdDistrict'] == district, ['X', 'Y']])
    test.loc[test['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
        test.loc[test['PdDistrict'] == district, ['X', 'Y']])

train_gdf = create_gdf(train)

데이터의 이상치와 중복을 처리한 후에 변수들을 검사합니다.


### Dates & Day of the week  
데이터들은 2003/1/1부터 2015/5/13까지(그리고 월요일부터 일요일까지) 균등하게 분포되어 있고 이전에 언급한 것처럼  training과 testing 데이터로 나뉘어져 있다. 이 데이터들에 대해서 어떠한 이상도 발견하지 못했다.
범죄 발생빈도의 중앙값(median)은 389이고 standard deviation(표준편차)는 48.51이다.


In [ ]:
col = sns.color_palette()

train['Date'] = train.Dates.dt.date
train['Hour'] = train.Dates.dt.hour

plt.figure(figsize=(10, 6))
data = train.groupby('Date').count().iloc[:, 0]
sns.kdeplot(data=data, shade=True)
plt.axvline(x=data.median(), ymax=0.95, linestyle='--', color=col[1])
plt.annotate(
    'Median: ' + str(data.median()),
    xy=(data.median(), 0.004),
    xytext=(200, 0.005),
    arrowprops=dict(arrowstyle='->', color=col[1], shrinkB=10))
plt.title(
    'Distribution of number of incidents per day', fontdict={'fontsize': 16})
plt.xlabel('Incidents')
plt.ylabel('Density')
plt.legend().remove()
plt.show()

또한 일주일 내내 범죄 발생 빈도의 심각한 deviation은 없다. 그러므로 DayOfWeek 변수가 예측에 중요한 역할을 할 것으로 기대히지 않는다.

In [ ]:
data = train.groupby('DayOfWeek').count().iloc[:, 0]
data = data.reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
    'Sunday'
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Incidents per Weekday', fontdict={'fontsize': 16})
plt.xlabel('Weekday')
plt.ylabel('Incidents (%)')

plt.show()

### Category  
도난(Larceny/Theft)(19.91%), 비범죄(Non/Criminal)(10.50%), 그리고 폭행(8.77%)순으로 정리되는 경찰서에서 정리하는 39개 타입의 Category가 있습니다.


In [ ]:
data = train.groupby('Category').count().iloc[:, 0].sort_values(
    ascending=False)
data = data.reindex(np.append(np.delete(data.index, 1), 'OTHER OFFENSES'))

plt.figure(figsize=(10, 10))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        (data.values / data.values.sum()) * 100,
        data.index,
        orient='h',
        palette="Reds_r")

plt.title('Incidents per Crime Category', fontdict={'fontsize': 16})
plt.xlabel('Incidents (%)')

plt.show()

### Police District  
범죄 발생수는 가장 많은 범죄가 발생한 Sourthern district(17.8%), 다음에 Mission(13.67%)과 Northern(12.00%) 순으로 도시의 다른 지구들 사이에 현격한 차이가 있다.

In [ ]:
# Downloading the shapefile of the area 
url = 'https://data.sfgov.org/api/geospatial/wkhw-cjsf?method=export&format=Shapefile'
with urllib.request.urlopen(url) as response, open('pd_data.zip', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
# Unzipping it
with zipfile.ZipFile('pd_data.zip', 'r') as zip_ref:
    zip_ref.extractall('pd_data')
# Loading to a geopandas dataframe
for filename in os.listdir('./pd_data/'):
    if re.match(".+\.shp", filename):
        pd_districts = gpd.read_file('./pd_data/'+filename)
        break
# Defining the coordinate system to longitude/latitude
pd_districts.crs={'init': 'epsg:4326'}

# Merging our train dataset with the geo-dataframe
pd_districts = pd_districts.merge(
    train.groupby('PdDistrict').count().iloc[:, [0]].rename(
        columns={'Dates': 'Incidents'}),
    how='inner',
    left_on='district',
    right_index=True,
    suffixes=('_x', '_y'))

# Transforming the coordinate system to Spherical Mercator for
# compatibility with the tiling background
pd_districts = pd_districts.to_crs({'init': 'epsg:3857'})

# Calculating the incidents per day for every district
train_days = train.groupby('Date').count().shape[0]
pd_districts['inc_per_day'] = pd_districts.Incidents/train_days

# Ploting the data
fig, ax = plt.subplots(figsize=(10, 10))
pd_districts.plot(
    column='inc_per_day',
    cmap='Reds',
    alpha=0.6,
    edgecolor='r',
    linestyle='-',
    linewidth=1,
    legend=True,
    ax=ax)

def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    """Function that add the tile background to the map"""
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))

# Adding the background
add_basemap(ax, zoom=11, url=ctx.sources.ST_TONER_LITE)

# Adding the name of the districts
for index in pd_districts.index:
    plt.annotate(
        pd_districts.loc[index].district,
        (pd_districts.loc[index].geometry.centroid.x,
         pd_districts.loc[index].geometry.centroid.y),
        color='#353535',
        fontsize='large',
        fontweight='heavy',
        horizontalalignment='center'
    )

ax.set_axis_off()
plt.show()

### Address  
Address는, 문자열 데이터로서 예측에 사용하기 위해서는 고난이도의 솜씨가 필요하다. 이 프로젝트에서는 대신에, 범죄가 빌딩에서 발생했는지 아니면 건물 안에서 발생했는지의 정보를 추출하는 데 사용할 것이다.
### X - 경도 Y - 위도  
이전에 좌표값이 도시 경계 안에 속하는지 테스트를 진행하였다. 경도값에는 오류가 없지만 위도에는 북극에 해당하는 90에 가까운 값들이 있다.

## Exploratory Visualization(탐색적 시각화)
프로젝트 성명에 근거하여, 시간과 장소에 따른 범죄 타입 각각에 대한 확률을 예측해야 한다. 즉 이 변수들의 중요성을 시각화하기 위해 두 개의 다이어그램을 제시한다.<br>
첫번째 것은 9 개의 랜덤한 범죄 카테고리의 지리적 밀도를 표현한다. <br>대부분 범죄 발생 위치가 도시의 북동쪽에 있지만, 나머지 도시에서 각 볌죄는 서로 다른 밀도를 가진다.<br>이 사실은 위치(좌표/경찰시 지구)가 분석 및 예측에 중요한 요소임을 나타낸다.

In [ ]:
crimes = train['Category'].unique().tolist()
crimes.remove('TREA')

pd_districts = pd_districts.to_crs({'init':'epsg:4326'})
sf_land = pd_districts.unary_union
sf_land = gpd.GeoDataFrame(gpd.GeoSeries(sf_land), crs={'init':'epsg:4326'})
sf_land = sf_land.rename(columns={0:'geometry'}).set_geometry('geometry')

fig, ax = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12,12))
for i , crime in enumerate(np.random.choice(crimes, size=9, replace=False)):
    data = train_gdf.loc[train_gdf['Category'] == crime]
    ax = fig.add_subplot(3, 3, i+1)
    gplt.kdeplot(data,
                 shade=True,
                 shade_lowest=False,
                 clip = sf_land.geometry,
                 cmap='Reds',
                 ax=ax)
    gplt.polyplot(sf_land, ax=ax)
    ax.set_title(crime) 
plt.suptitle('Geographic Density of Different Crimes')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

두번째 다이어그램은 5가지 카테고리에 대한 시간당 평균 범죄 수를 나타낸다. 이것은 여러 시간대에 다른 범죄들이 서로 다른 빈도들을 가진다는 증거이다.<br>
예를 들어, 매춘은 저녁시간부터 밤새 발생하고, 도박은 늦은 밤부터 아침까지 발생하며 강도사건은 아침 일찍부터 오후 시간에 많이 발생한다.<br>
이전과 마찬가지로, 시간 역시 중요한 역할을 할 것이라는 명백한 증거이다.

In [ ]:
data = train.groupby(['Hour', 'Date', 'Category'],
                     as_index=False).count().iloc[:, :4]
data.rename(columns={'Dates': 'Incidents'}, inplace=True)
data = data.groupby(['Hour', 'Category'], as_index=False).mean()
data = data.loc[data['Category'].isin(
    ['ROBBERY', 'GAMBLING', 'BURGLARY', 'ARSON', 'PROSTITUTION'])]

sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(14, 4))
ax = sns.lineplot(x='Hour', y='Incidents', data=data, hue='Category')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=6)
plt.suptitle('Average number of incidents per hour')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## Algorithms and Techniques  
이 문제는 일반적인 멀티클래스 분류 문제이며 문제 해결에 사용할 수 있는 여러 가지 알고리즘 카테고리가 있다.<br>
처음에는 기본적인 feature engineering과 기본 파라메터를 사용한 SGD, kNN, Ensemble methods(Random Forest & Adaboost), Boosting Algorithm(XGBoost & LightGBM) 모델들을 평가해서 좋은 시작점으로 삼을 수 있는지 확인한다.

| **Algorithm**                   | **Parameters**                                        | **Logloss**  |
|:---------------------------:|:-------------------------------------------------:|:--------:|
| Stochastic Gradient Descent | Default Scikit-Learn Parameters (with 'log' loss) |  2.86631 |
| K-Nearest Neighbors         | Default Scikit-Learn Parameters                   | 23.29263 |
| Random Forest               | Default Scikit-Learn Parameters                   |  2.92716 |
| AdaBoost                    | Default Scikit-Learn Parameters                   |  3.58856 |
| XGBoost                     | Default Scikit-Learn Parameters                   |  2.91656 |
| LIghtGBM                    | Default Scikit-Learn Parameters                   |  2.98336 |  

SGD가 가장 좋은 초기 결과를 기록했지만, 긴 하이퍼파라메터 조정 후에는 2.54503 임계값을 넘을 수 없었다.

마지막으로, 3.0 이하의 점수를 얻은 알고리즘 중에서, 하이퍼파라메터 튜닝에 효율적이고 다재능함이 있어  LightGBM을 사용하기로 결정하였다.<br>
LightGBM은 decision tree boosting 알고리즘으로 연속된 feature(attribute) 값을 이산 bin으로 버킷화하는 히스토그램 기반의 알고리즘을 사용한다. <br>이 테크닉은 트레이닝 속도를 올리고 메모리 사용량을 줄여준다. <br>이 알고리즘은 다음과 같이 동작한다.

1. decision tree를 훈련시킨다.
2. 모델을 평가한다.
3. 잘못 분류된 sample의 가중치를 증가시킨다.
4. 자랄 tree로 최대 loss가 있는 leaf를 선택한다.
5. tree를 자라게 한다.
6. step 2로 돌아간다. ![lightgbm](https://lightgbm.readthedocs.io/en/latest/_images/leaf-wise.png) 

## Benchmark  
There are two types of benchmarks we need to set. The first will be a naive prediction. This prediction will be a baseline score to compare with our model’s score to evaluate if we have any significant progress.  

In a Multiclass Classification, the best way to calculate the baseline is by assuming that the probability of each category equals its average frequency in the train set. The frequency can be calculated easily by dividing the sum of incidents of each category by the number of rows of the training set.


In [ ]:
naive_vals = train.groupby('Category').count().iloc[:,0]/train.shape[0]
n_rows = test.shape[0]

submission = pd.DataFrame(
    np.repeat(np.array(naive_vals), n_rows).reshape(39, n_rows).transpose(),
    columns=naive_vals.index)

The baseline calculated this way is 2.68015. (Details in [SF-Crime Analysis & Prediction (Naive Prediction)](https://www.kaggle.com/yannisp/sf-crime-analysis-prediction-naive-prediction) Notebook. We can notice that this baseline is already lower than the initial score of our classifiers.  

Another critical benchmark is usually the ‘Human Performance’, as a proxy for the Bayes error rate. The specific problem does not belong to a field that humans excel (like computer vision or NLP), so as a proxy for the  [Bayes error rate](https://en.wikipedia.org/wiki/Bayes_error_rate), we will use the score of the best kernel so far which is [initial benchmark need tuning](https://www.kaggle.com/sergeylebedev/initial-benchmark-need-tuning) by the user [Sergey Lebedev](https://www.kaggle.com/sergeylebedev) with score 2.29318. 

The small distance between the baseline score and the Bayes error rate indicate that this is a hard problem with a low margin of improvement.  
# Methodology
## Data Preprocessing
### Data Wrangling
Following the methodology described in the Problem Statement, we identified 2323 duplicate values and 67 wrong latitudes. The duplicates removed and the outliers imputed.  
### Feature Engineering
Then, we created additional features. More specifically:
* From the ‘Dates’ field, we extracted the Day, the Month, the Year, the Hour, the Minute, the Weekday, and the number of days since the first day in the data.
* From the ‘Address’ field we extracted if the incident has taken place in a crossroad or on a building block.




In [ ]:
def feature_engineering(data):
    data['Date'] = pd.to_datetime(data['Dates'].dt.date)
    data['n_days'] = (
        data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False)
    
    data.drop(columns=['Dates','Date','Address'], inplace=True)
        
    return data

In [ ]:
train = feature_engineering(train)
train.drop(columns=['Descript','Resolution'], inplace=True)
test = feature_engineering(test)
train.head()

### Feature Scaling
Deciding to continue with a tree-based algorithm there was no need for scaling on the final dataset.
### Feature Selection
After the feature engineering described above, we ended up with 11 features. To identify if any of them increased the complexity of the model without adding significant gain to the model, we used the method of Permutation Importance.  

The idea is that the importance of a feature can be measured by looking at how much the loss decreases when a feature is not available. To do that we can remove each feature from the dataset, re-train the estimator and check the impact. Doing this would require re-training an estimator for each feature, which can be computationally intensive. Instead, we can replace it with noise by shuffle values for a feature.  

The implementation of the above technique showed that there is no need for any feature removal since all of them have a positive impact in the dataset.


In [ ]:
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])
le2 = LabelEncoder()
y = le2.fit_transform(train.pop('Category'))

train_X, val_X, train_y, val_y = train_test_split(train, y)

model =LGBMClassifier(objective='multiclass', num_class=39).fit(train_X, train_y)

perm = PermutationImportance(model).fit(val_X, val_y)
eli5.show_weights(perm, feature_names=val_X.columns.tolist())

## Building the Initial Model
To build the model we used the LightGBM’s Python API.
First we created the dataset by combining the features, the target and declaring the PdDistrict as a categorical variable using ‘lightgbm.Dataset()`.  

Then we used Cross-Validation with early stopping (10 rounds) and parameters:  
* Objective = ‘'multiclass',  
* 'Metric = ‘multi_logloss',  
* 'Num_class = 39  

The above setup achieved 2.46799  cross-validation score after 23 epochs and 2.49136 on the testing set.  
[SF-Crime Analysis & Prediction (Base Model)](https://www.kaggle.com/yannisp/sf-crime-analysis-prediction-base-model/notebook?scriptVersionId=9334889) 
![base_model](https://i.imgur.com/AcJHphZ.png)
## Refinement  
Instead of the most popular methods of Exhaustive Grid Search and Randomized Parameter Optimization, we selected another more efficient way to tune the hyperparameters of the algorithm; **Bayesian optimization**.  

The problem with the two techniques mentioned above is that they do not use previous results to pick the next input values. Instead, Bayesian optimization, also called Sequential Model-Based Optimization (SMBO), implements this idea by building a probability model of the objective function that maps input values to a probability of a loss: p (loss | input values). The probability model, (also called the surrogate or response surface), is easier to optimize than the actual objective function. Bayesian methods select the next values to evaluate by applying a criterion (usually Expected Improvement) to the surrogate. The concept is to limit the evaluations of the objective function by spending more time choosing the next values to try.  

To conclude to the final model, we used five folds Cross-Validation for 100 epochs and early stopping with Bayesian Optimization. Also, we created a custom callback function so we can write proper logs that can be read by Tensorboard. This way we were able to monitor the validation process in real time.  

The above was up to some point an iterative process. We run the optimization process until we noticed in Tensorboard that the models converge. Then we stopped and evaluated the results and move to the next iteration. (example of the process [here](https://www.kaggle.com/yannisp/sf-crime-analysis-prediction-optimiz-ex)) 

First we optimized a few basic hyperparameters including:
* `Boosting` selection between gbdt and dart
* `Max_delta_step` uniformly in the range  [0, 2]
* `Min_data_in_leaf` uniformly in the range [10,30]
* `Num_leaves` uniformly in the range [20,40]  

After the model converged, a second round of tuning followed:
* `Boosting`: gbdt
* `Max_delta_step` uniformly in the range  [0.5, 2.5]
* `Min_data_in_leaf` uniformly in the range [10, 25]
* `Num_leaves` uniformly in the range [20, 45]
* `Max_bin` uniformly in the range [200, 500],
* `Learning_rate` uniformly in the range [0.1, 2]

Finally we concluded to the following hyperparameters:  
* `Boosting`: gbdt
* `Max_delta_step`: 0.9
* `Min_data_in_leaf`:  21
* `Num_leaves`: 41
* `Max_bin`: 465,
* `Learning_rate`: 0.4  

In the following figure, we present the performance of the best model from each step of optimization.  
![](https://i.imgur.com/grRIpi5.png?1)


## Building the final model

In [ ]:
# Loading the data
train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')

# Data cleaning
train.drop_duplicates(inplace=True)
train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

imp = SimpleImputer(strategy='mean')

for district in train['PdDistrict'].unique():
    train.loc[train['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(
        train.loc[train['PdDistrict'] == district, ['X', 'Y']])
    test.loc[test['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
        test.loc[test['PdDistrict'] == district, ['X', 'Y']])
train_data = lgb.Dataset(
    train, label=y, categorical_feature=['PdDistrict'], free_raw_data=False)

# Feature Engineering
train = feature_engineering(train)
train.drop(columns=['Descript','Resolution'], inplace=True)
test = feature_engineering(test)

# Encoding the Categorical Variables
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

le2 = LabelEncoder()
X = train.drop(columns=['Category'])
y= le2.fit_transform(train['Category'])

# Creating the model
train_data = lgb.Dataset(
    X, label=y, categorical_feature=['PdDistrict'])

params = {'boosting':'gbdt',
          'objective':'multiclass',
          'num_class':39,
          'max_delta_step':0.9,
          'min_data_in_leaf': 21,
          'learning_rate': 0.4,
          'max_bin': 465,
          'num_leaves': 41
         }

bst = lgb.train(params, train_data, 100)

predictions = bst.predict(test)

# Submitting the results
submission = pd.DataFrame(
    predictions,
    columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),
    index=test.index)
submission.to_csv(
    'LGBM_final.csv', index_label='Id')

# Model Evaluation and Validation
The final model scored 2.25697 on the training set which is 16% lower from the naive prediction (2.68015)  and 2% better than the benchmark. Taking into account the low margin between the naive and the benchmark we knew that we would probably have a small improvement.  

That being said, we could say that the results are satisfactory.  

Based on the Permutation Importance analysis we performed before, the model should be susceptible to changes in Minute and the coordinates and less sensitive to changes in Day, Year or Day of the week.  

Indeed, by removing the ‘Minute’ feature from the dataset, we had an increase of loss to 2.53743 and by removing the ‘DayOfWeek’ feature the loss increased to 2.25900.  

The Permutation importance is a great tool to understand **how much** a specific feature affect our prediction but it does not tell us anything about **the direction** it affects it. We can solve this issue and understand even deeper our model by using Partial Dependencies. In other words, if for an incident we change only the value of one feature how will this affect the probability of each crime category?  

As an example, we can evaluate how the Hour affects the probabilities of three different crimes. We can see that the hour does not affect the probability for BRIBERY (class 3). In contrast, during the night the probability for BURGLARY (class 4) increases (up to 2%), and during the day the probability for DISORDERLY CONDUCT (class 5) decreases.  

We can conclude that the model is aligned with our intuition.



In [ ]:
model = LGBMClassifier(**params).fit(X, y, categorical_feature=['PdDistrict'])

pdp_Pd = pdp.pdp_isolate(
    model=model,
    dataset=X,
    model_features=X.columns.tolist(),
    feature='Hour',
    n_jobs=-1)

pdp.pdp_plot(
    pdp_Pd,
    'Hour',
    ncols=3)
plt.show()

# Conclusion
## Free-Form Visualization
An interesting visualization would be to depict how each feature affects a specific prediction. Insights like this are possible with the use of the SHAP library. 

SHAP (SHapley Additive exPlanations) is a unified approach to explain the output of any machine learning model. SHAP connects game theory with local explanations, uniting several previous methods and representing the only possible consistent and locally accurate additive feature attribution method based on expectations (see the [SHAP NIPS paper](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions) for details).  

As an example let’s select a row from the testing dataset:

In [ ]:
data_for_prediction = test.loc[[846262]]
data_for_prediction

This incident has taken place in 03:30 in the night in a block. As we saw in the Partial Dependencies graphs before, BURGLARY has a higher probability and burglaries happen by definition in blocks. Let’s see if our model aligns with our intuition.

In [ ]:
shap.initjs()

# Create object that can calculate shap values
explainer = shap.TreeExplainer(model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)

shap.force_plot(explainer.expected_value[4], shap_values[4], data_for_prediction, link='logit')

We can see that there is a 10% probability for BURGLARY and that this is mostly increased  because it takes place to a block (not on a crossroad) and from the time (hour and minute). Both of these are aligned again with our intuition, making us more confident about the validity of our model.

## Reflection
As described in the previous sections a full cycle data processing have been followed and lead us to a satisfactory prediction model.  

The most challenging part was that due to the nature of the features, there was a little room for feature engineering. For this reason, we had to be creative and use advanced techniques during the hyperparameter optimization to make a difference.  
This is a hard problem to solve with a heavily unbalanced dataset and the unpredictability (up to some point) of the “human factor”.  
## Improvements
We are sure there is space for improvement. Two additional techniques we would like to implement if there was the necessary time would be:  

* Create **ordinal representations** for the features that present a kind of cyclicity (Month, Weekday, Hour, Minute). The reasoning behind this is that if we take the Hour as an example, the default representation implies that 23 and 00 (midnight) are 23 “units” away although in reality, they are 1 “unit” apart. A way to solve it is to imagine the hour in a real clock and take their projections on the axes passing from the center of the clock. This way the distance between 23 and 00 is the same as between 00 and 01. We can achieve this with functions like Hx = sin(2\*π\*H/23) & Hy = cos(2\*π\*H/23) for the hour and accordingly for the rest.    

* Use **embeddings** or any other text processing technique, for the addresses. By extracting if the incident has happened to a block or a crossroad we have extracted the minimum gain from this feature, and maybe there are some patterns to exploit and give us even better score.